# 計算word paragraphs，並印出

In [ ]:
import docx
Doc = docx.Document(r"C:\Users\jackiehuang\Desktop\python\docx\test_1.docx")

print("檔案內含段落數：",len(Doc.paragraphs),"\n")

testList = []
for text in Doc.paragraphs:
    testList.append(text)

for pg in testList:
    print(pg.text)

# python-docx範例word

In [ ]:
from docx import Document
from docx.shared import Inches

document = Document()

document.add_heading('Document Title', 0)

p = document.add_paragraph('A plain paragraph having some ')
p.add_run('bold').bold = True
p.add_run(' and some ')
p.add_run('italic.').italic = True

document.add_heading('Heading, level 1', level=1)
document.add_paragraph('Intense quote', style='Intense Quote')

document.add_paragraph(
    'first item in unordered list', style='List Bullet'
)
document.add_paragraph(
    'first item in ordered list', style='List Number'
)

document.add_picture('lena.png', width=Inches(1.25))

records = (
    (3, '101', 'Spam'),
    (7, '422', 'Eggs'),
    (4, '631', 'Spam, spam, eggs, and spam')
)

table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
for qty, id, desc in records:
    row_cells = table.add_row().cells
    row_cells[0].text = str(qty)
    row_cells[1].text = id
    row_cells[2].text = desc

document.add_page_break()

document.save('test_2.docx')

# Excel 表格轉為word表格

In [ ]:
import docx
import openpyxl
import random
from docx.oxml.ns import qn
from docx.shared import Pt, RGBColor
from win32com.client import Dispatch

'''
如果excel資料表中有代公式，處理前要先用win32com開啟儲存一次，
不然load_workbook(..., data_only=True) 讀出結果會是None或計算好的值，
後面save()會儲存值，公式會消失。
data_only=False 則只會顯示公式跟儲存公式
'''
xlApp = Dispatch("Excel.Application")
xlApp.Visible = False
#要放絕對路徑，前面+r
xlBook = xlApp.Workbooks.Open(r"C:\Users\jackiehuang\Desktop\python\docx\test_3.xlsx")
xlBook.Save()
xlBook.Close()

wb=openpyxl.load_workbook("test_3.xlsx", data_only=False)
sheet=wb["工作表1"]
doc=docx.Document()

maxrow=sheet.max_row
maxcol=sheet.max_column

table=doc.add_table(rows=maxrow,cols=maxcol, style = 'Table Grid')
for i in range(maxrow):
    for j in range(maxcol):
        run = table.cell(i,j).paragraphs[0].add_run(str(sheet.cell(i+1,j+1).value))
        run.font.color.rgb = RGBColor(int(random.random() * 255), int(random.random() * 255), int(random.random() * 255))

wb.save("test_3.xlsx")
doc.save("test_3.docx")

# Excel 畫圖

In [ ]:
import openpyxl
from openpyxl.chart import BarChart, Series, Reference

wb=openpyxl.load_workbook("test_3.xlsx")
sheet=wb["工作表1"]

result=openpyxl.Workbook()
result.create_sheet(index=0,title="工資圖表")
test=result["工資圖表"]

maxrow=sheet.max_row
test.cell(row=1, column=1).value = sheet.cell(row=1, column=2).value
test.cell(row=1, column=2).value = sheet.cell(row=1, column=6).value
for i in range(2,maxrow+1):
    test.cell(row=i,column=1).value=sheet.cell(row=i,column=2).value
    test.cell(row=i, column=2).value = float(sheet.cell(row=i, column=6).value)


chart1 = BarChart()
chart1.type = "col"
chart1.style = 10
chart1.title = "工資圖表"
chart1.y_axis.title = '工資'
chart1.x_axis.title = '姓名'
data = Reference(test, min_col=2, max_col=2,min_row=1, max_row=maxrow) # 条形图中的数字数据（看见的矩形）
cats = Reference(test, min_col=1, min_row=2, max_row=maxrow) # 横坐标上的姓名
chart1.add_data(data, titles_from_data=True)
chart1.set_categories(cats)
chart1.shape = 4

test.add_chart(chart1, "A10")


wb.save("test_3.xlsx")
result.save("test_4.xlsx")

# 比較excel表格是否相同

In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_excel('test_5a.xlsx')
df2 = pd.read_excel('test_5b.xlsx')
comparison_values = df1.values == df2.values
print(comparison_values)